In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests

In [27]:
def fetch_drug_info(drug_name):
    # Step 1: Search for the drug by name
    search_url = f"https://rxnav.nlm.nih.gov/REST/rxcui.json?name={drug_name}"
    search_response = requests.get(search_url)
    search_data = search_response.json()

    # Extract the RxCUI identifier for the drug
    rxcui = search_data["idGroup"].get("rxnormId", [None])[0]
    if not rxcui:
        return "Drug name not found."

    # Step 2: Retrieve drug details using the RxCUI
    details_url = f"https://rxnav.nlm.nih.gov/REST/rxcui/{rxcui}/allrelated.json"
    details_response = requests.get(details_url)
    details_data = details_response.json()

    # Initialize containers for the names
    branded_names = []
    generic_names = []

    # Extract details for branded and generic names
    for concept_group in details_data["allRelatedGroup"]["conceptGroup"]:
        if concept_group["tty"] == "BN":
            branded_names.extend(
                [concept["name"] for concept in concept_group["conceptProperties"]]
            )
        elif concept_group["tty"] == "IN":
            generic_names.extend(
                [concept["name"] for concept in concept_group["conceptProperties"]]
            )

    return {"Branded Names": branded_names, "Generic Names": generic_names}


# drug_list
drugs = ["acetaminophen", "ibuprofen", "aspirin", "metformin", "simvastatin"]

# Fetch drug information for each drug
for drug in drugs:
    drug_info = fetch_drug_info(drug)
    print(f"Drug: {drug}")
    print(f"Branded Names: {drug_info['Branded Names']}")
    print(f"Generic Names: {drug_info['Generic Names']}")
    print("\n")

Drug: acetaminophen
Branded Names: ['Little Fevers', 'Bactimicina', 'Ofirmev', 'Pamprin Max Formula', 'Pamprin Multi-Symptom', 'Pamprin Cramp Formula', 'Percogesic Reformulated Jan 2011', 'Premsyn PMS', 'Panadol PM', 'CounterAct Pain', 'Capacet', 'Nortemp', 'Acetadryl', 'Sedalmex', 'Ringl', 'Pharbetol', 'Arthriten Inflammatory Pain', 'Backaid', 'Cetafen', 'Exaprin', 'Backaid IPF', 'Lortab', 'Primlev', 'Verdrocet', 'Excedrin Mild Headache', 'Excedrin PM Triple Action', 'Ninjacof A', 'Vanatol', 'Fioricet', 'Sed-Max', 'Panadol Extra', 'Arthriten Max', 'Apadaz', 'Panadol', 'Tylenol', 'Nalocet', 'Dvorah', 'Anacin AF', 'Bupap', 'Endocet', 'Esgic', 'Excedrin', 'Excedrin PM', 'Feverall', 'Fioricet with Codeine', 'Legatrin PM', 'Mapap', 'Marten-Tab', 'Norco', 'Redutemp', 'Repan', 'Saleto', 'Tempra Quicklets', 'Tencon', 'Tycolene', 'Tylenol PM', 'Tylenol with Codeine', 'Unisom with Pain Relief', 'Vanquish', 'Zebutal', 'Acephen', 'Vtol', 'Allzital', 'Prolate', 'Combogesic', 'Contragesic', 'Relage

In [7]:
import requests
import pandas as pd


def fetch_drug_info(drug_name):
    # Step 1: Search for the drug by name
    search_url = f"https://rxnav.nlm.nih.gov/REST/rxcui.json?name={drug_name}"
    search_response = requests.get(search_url)
    search_data = search_response.json()

    # Extract the RxCUI identifier for the drug
    rxcui = search_data["idGroup"].get("rxnormId", [None])[0]
    if not rxcui:
        return pd.DataFrame()

    # Step 2: Retrieve drug details using the RxCUI
    details_url = f"https://rxnav.nlm.nih.gov/REST/rxcui/{rxcui}/allrelated.json"
    details_response = requests.get(details_url)
    details_data = details_response.json()

    # Initialize containers for the names
    data_pairs = []

    # Extract details for branded and generic names
    generic_names = []
    branded_names = []

    for concept_group in details_data["allRelatedGroup"]["conceptGroup"]:
        if concept_group["tty"] == "IN":
            generic_names = [
                concept["name"] for concept in concept_group["conceptProperties"]
            ]
        elif concept_group["tty"] == "BN":
            branded_names = [
                concept["name"] for concept in concept_group["conceptProperties"]
            ]

    # Pair each brand with the generic name
    for brand in branded_names:
        for generic in generic_names:
            data_pairs.append({"Generic Name": generic, "Brand Name": brand})

    # Create DataFrame from pairs
    return pd.DataFrame(data_pairs)


# drug_list
drugs = [
    "acetaminophen",
    "ibuprofen",
    "aspirin",
    "metformin",
    "simvastatin",
    "Abemaciclib",
    "Keytruda",
]

# Initialize an empty DataFrame to collect all results
all_drug_info = pd.DataFrame()

# Fetch drug information for each drug and concatenate to the DataFrame
for drug in drugs:
    drug_info = fetch_drug_info(drug)
    all_drug_info = pd.concat([all_drug_info, drug_info], ignore_index=True)

print(all_drug_info)

      Generic Name             Brand Name
0    acetaminophen          Little Fevers
1    acetaminophen            Bactimicina
2    acetaminophen                Ofirmev
3    acetaminophen    Pamprin Max Formula
4    acetaminophen  Pamprin Multi-Symptom
..             ...                    ...
164    simvastatin               Flolipid
165    simvastatin                  Zocor
166    simvastatin                Vytorin
167    abemaciclib               Verzenio
168  pembrolizumab               Keytruda

[169 rows x 2 columns]


In [22]:
import requests
import pandas as pd


def fetch_drug_info(max_rxcui):
    data_pairs = []  # Initialize container for all data pairs

    for rxcui in range(1, max_rxcui + 1):
        try:
            # Step 2: Retrieve drug details using the RxCUI
            details_url = (
                f"https://rxnav.nlm.nih.gov/REST/rxcui/{rxcui}/allrelated.json"
            )
            details_response = requests.get(details_url)

            # Check if the response was successful
            if details_response.status_code != 200:
                continue  # Skip this iteration if the request failed

            details_data = details_response.json()

            # Initialize containers for the names
            generic_names = []
            branded_names = []

            for concept_group in details_data.get("allRelatedGroup", {}).get(
                "conceptGroup", []
            ):
                if "conceptProperties" in concept_group:
                    if concept_group["tty"] == "IN":
                        generic_names = [
                            concept["name"]
                            for concept in concept_group["conceptProperties"]
                        ]
                    elif concept_group["tty"] == "BN":
                        branded_names = [
                            concept["name"]
                            for concept in concept_group["conceptProperties"]
                        ]

            # Pair each brand with the generic name
            for brand in branded_names:
                for generic in generic_names:
                    data_pairs.append({"Generic Name": generic, "Brand Name": brand})

        except Exception as e:
            print(f"Error with RxCUI {rxcui}: {e}")
            continue

    # Create DataFrame from pairs
    return pd.DataFrame(data_pairs)


# Example usage: fetch information for the first 100 possible RxCUIs
drug_info = fetch_drug_info(100)
print(drug_info)

           Generic Name                            Brand Name
0         bromocriptine                              Parlodel
1                 mesna                                Mesnex
2      docosahexaenoate                             Algal-900
3      docosahexaenoate                               Natelle
4      docosahexaenoate  Ocean Blue Nutritionals Omega-3 1450
5   4-aminobenzoic acid                              SwineAid
6     eicosapentaenoate                               Natelle
7     eicosapentaenoate  Ocean Blue Nutritionals Omega-3 1450
8   5-hydroxytryptophan                              Somnicin
9           ticlopidine                                Ticlid
10  6-aminocaproic acid                                Amicar


In [23]:
import requests
import pandas as pd


def fetch_drug_info(max_rxcui):
    data_pairs = []  # Initialize container for all data pairs

    for rxcui in range(1, max_rxcui + 1):
        try:
            # Step 2: Retrieve drug details using the RxCUI
            details_url = (
                f"https://rxnav.nlm.nih.gov/REST/rxcui/{rxcui}/allrelated.json"
            )
            details_response = requests.get(details_url)

            # Check if the response was successful
            if details_response.status_code != 200:
                continue  # Skip this iteration if the request failed

            details_data = details_response.json()

            # Initialize containers for the names
            generic_names = []
            branded_names = []

            for concept_group in details_data.get("allRelatedGroup", {}).get(
                "conceptGroup", []
            ):
                if "conceptProperties" in concept_group:
                    if concept_group["tty"] == "IN":
                        generic_names = [
                            concept["name"]
                            for concept in concept_group["conceptProperties"]
                        ]
                    elif concept_group["tty"] == "BN":
                        branded_names = [
                            concept["name"]
                            for concept in concept_group["conceptProperties"]
                        ]

            # Pair each brand with the generic name and include the RxCUI
            for brand in branded_names:
                for generic in generic_names:
                    data_pairs.append(
                        {"RxCUI": rxcui, "Generic Name": generic, "Brand Name": brand}
                    )

        except Exception as e:
            print(f"Error with RxCUI {rxcui}: {e}")
            continue

    # Create DataFrame from pairs
    return pd.DataFrame(data_pairs)


# Example usage: fetch information for the first 100 possible RxCUIs
drug_info = fetch_drug_info(100)
print(drug_info)

    RxCUI         Generic Name                            Brand Name
0      38        bromocriptine                              Parlodel
1      44                mesna                                Mesnex
2      73     docosahexaenoate                             Algal-900
3      73     docosahexaenoate                               Natelle
4      73     docosahexaenoate  Ocean Blue Nutritionals Omega-3 1450
5      74  4-aminobenzoic acid                              SwineAid
6      90    eicosapentaenoate                               Natelle
7      90    eicosapentaenoate  Ocean Blue Nutritionals Omega-3 1450
8      94  5-hydroxytryptophan                              Somnicin
9      97          ticlopidine                                Ticlid
10     99  6-aminocaproic acid                                Amicar
